In [2]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
np.random.seed(13)
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
import os
import cv2
from skimage.transform import *
from skimage.color import *

In [31]:
FILE_PATH = 'C://Users//yashb//Documents//Predictive//HW4//train//train'
images = []
ids = []
mappings = dict()
count = 0
n = 100
# parse through all files in the directory
for dirpath, dirs, files in os.walk(FILE_PATH):
    for filename in files:
        #if count == n:
            #break
        img_path = dirpath+'/'+filename
        img = cv2.imread(img_path)
        img = resize(img, (128, 128))
        #ids.append(filename)
        #images.append(img)
        mappings[filename] = img
        count += 1 

In [32]:
image_data = pd.DataFrame(mappings.items(), columns = ['file','img'])
image_data

,file,img
0,cat.0.jpg,"[[[0.3429534313725487, 0.6449142156862742, 0.7..."
1,cat.1.jpg,"[[[0.1568627450980392, 0.17022058823529368, 0...."
2,cat.10.jpg,"[[[0.16253578335630964, 0.12828075932521243, 0..."
3,cat.100.jpg,"[[[0.8492187499999995, 0.8688265931372545, 0.8..."
4,cat.1000.jpg,"[[[0.29285852768841913, 0.43795656690410545, 0..."
...,...,...
24995,dog.9995.jpg,"[[[0.4156862745098039, 0.5607843137254902, 0.5..."
24996,dog.9996.jpg,"[[[0.2921782848881723, 0.4076758291207093, 0.5..."
24997,dog.9997.jpg,"[[[0.1590992647058827, 0.17086397058823563, 0...."
24998,dog.9998.jpg,"[[[0.9860753676470593, 0.9921568627450981, 0.9..."


In [33]:
image_data['label'] = image_data['file'].apply(lambda x :x.split('.')[0])

In [34]:
image_data.head()

,file,img,label
0,cat.0.jpg,"[[[0.3429534313725487, 0.6449142156862742, 0.7...",cat
1,cat.1.jpg,"[[[0.1568627450980392, 0.17022058823529368, 0....",cat
2,cat.10.jpg,"[[[0.16253578335630964, 0.12828075932521243, 0...",cat
3,cat.100.jpg,"[[[0.8492187499999995, 0.8688265931372545, 0.8...",cat
4,cat.1000.jpg,"[[[0.29285852768841913, 0.43795656690410545, 0...",cat


In [35]:
image_data['label'] = image_data['label'].where(image_data['label'] == 'cat',1)
image_data['label'] = image_data['label'].where(image_data['label'] == 1,0)

In [36]:
images = list(image_data['img'])

In [40]:
image_data.to_csv('train_data.csv', index = False)

In [70]:
image_data = pd.read_csv('train_data.csv')

In [71]:
image_data.head()

,file,img,label
0,cat.0.jpg,[[[0.34295343 0.64491422 0.79785539]\n [0.362...,0
1,cat.1.jpg,[[[0.15686275 0.17022059 0.15294118]\n [0.168...,0
2,cat.10.jpg,[[[0.16253578 0.12828076 0.11083284]\n [0.133...,0
3,cat.100.jpg,[[[0.84921875 0.86882659 0.86490502]\n [0.853...,0
4,cat.1000.jpg,[[[0.29285853 0.43795657 0.54776049]\n [0.297...,0


In [43]:
#x_train, x_test, y_train, y_test = train_test_split(final['img'], final['label'], test_size=0.2, random_state=1)

In [44]:
#train_df = final.iloc[0:20000, :]
#test_df = final.iloc[20000:, :]

In [72]:
x_train = np.array(np.array(images) * 255, dtype = np.uint8)
y_train = image_data['label'].to_numpy()

#x_test = np.array(np.array(images[20000:]) * 255, dtype = np.uint8)
#y_test = test_df['label'].to_numpy()

In [73]:
BATCH_SIZE = 2000
BUFFER_SIZE = BATCH_SIZE * 2
AUTO = tf.data.AUTOTUNE
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train.astype(float)))
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTO)

#test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test.astype(float)))
#test_ds = test_ds.batch(BATCH_SIZE).prefetch(AUTO)

In [74]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomContrast

In [48]:
nb_epoch = 30
nb_filters = 5 # the number of filters
nb_pool = 2 # window size of pooling
nb_conv = 3 # window size of filter
img_rows, img_cols = 128, 128
nb_classes = 2

"""
model = Sequential()
model.add(Convolution2D(nb_filters, nb_conv, nb_conv, input_shape=(img_rows, img_cols, 3)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation("sigmoid"))
"""


model = tf.keras.models.Sequential([

# This is the first convolution
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(img_rows, img_cols, 3)),
tf.keras.layers.MaxPooling2D(2,2),
# The second convolution
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
# The third convolution
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

    
tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
    
tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
    
# Flatten the results to feed into a DNN
tf.keras.layers.Flatten(),
# 512 neuron hidden layer
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')])
    

    

"""
def get_model():
    image_inputs = tf.keras.Input((img_rows, img_cols , 3))
    
    resnet = ResNet50(include_top=False, pooling=None)
    
    image_x = resnet(RandomContrast(factor = 0.1)(RandomRotation(factor = 0.15)(image_inputs)))
    
    image_x = tf.keras.layers.GlobalAveragePooling2D()(image_x)

    output = tf.keras.layers.Dense(1)(image_x)
    
    model = tf.keras.Model(inputs=[image_inputs], outputs=[output])
    return model
    
model = get_model()
"""

'\ndef get_model():\n    image_inputs = tf.keras.Input((img_rows, img_cols , 3))\n    \n    resnet = ResNet50(include_top=False, pooling=None)\n    \n    image_x = resnet(RandomContrast(factor = 0.1)(RandomRotation(factor = 0.15)(image_inputs)))\n    \n    image_x = tf.keras.layers.GlobalAveragePooling2D()(image_x)\n\n    output = tf.keras.layers.Dense(1)(image_x)\n    \n    model = tf.keras.Model(inputs=[image_inputs], outputs=[output])\n    return model\n    \nmodel = get_model()\n'

In [49]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 63, 63, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_12 (Conv2D)          (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 14, 14, 64)      

In [50]:
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
model.fit(train_ds, epochs=10, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/10
13/13 [==============================] - 890s 68s/step - loss: 30.6942 - accuracy: 0.6034
Epoch 2/10
13/13 [==============================] - 851s 65s/step - loss: 1.1075 - accuracy: 0.3976
Epoch 3/10
13/13 [==============================] - 861s 66s/step - loss: 1.4908 - accuracy: 0.3502
Epoch 4/10
13/13 [==============================] - 879s 68s/step - loss: 0.7027 - accuracy: 0.3784
Epoch 5/10
13/13 [==============================] - 895s 69s/step - loss: 0.7075 - accuracy: 0.2908
Epoch 6/10
13/13 [==============================] - 855s 65s/step - loss: 0.6954 - accuracy: 0.4300
Epoch 7/10
13/13 [==============================] - 835s 64s/step - loss: 0.6937 - accuracy: 0.4810
Epoch 8/10
13/13 [==============================] - 1089s 85s/step - loss: 0.6936 - accuracy: 0.4964
Epoch 9/10
13/13 [==============================] - 3053s 234s/step - loss: 0.6935 - accuracy: 0.4997
Epoch 10/10
13/13 [==============================] - 3036s 233s/step - loss: 0.6936 - accuracy: 

In [51]:
FILE_PATH = 'C://Users//yashb//Documents//Predictive//HW4//test//test'

ids = []
test_images = []
mapping = dict()
count = 0 
n = 100
# parse through all files in the directory
for dirpath, dirs, files in os.walk(FILE_PATH):
    for filename in files:
        #if count == n:
            #break
        img_path = dirpath+'/'+filename
        img = cv2.imread(img_path)
        img = resize(img, (128, 128))
        #ids.append(filename.split('.')[0])
        #test_mappings[] = img
        #test_images.append(img)
        mapping[filename.split('.')[0]] = img
        count += 1

In [64]:
#test_data = pd.DataFrame(columns = ['id'])
#test_data['id'] = ids
#test_data['id'] = list(test_mappings.keys())
#test_data['img'] = list(test_mappings.values())
test_data = pd.DataFrame(mapping.items(), columns = ['id','img'])
test_data

,id,img
0,1,"[[[0.3803921568627451, 0.2567095588235295, 0.1..."
1,10,"[[[0.0196078431372549, 0.027450980392156862, 0..."
2,100,"[[[0.00784313725490196, 0.00784313725490196, 0..."
3,1000,"[[[0.4701286764705901, 0.6905464920343154, 0.6..."
4,10000,"[[[0.4489123774509831, 0.5104669907513809, 0.5..."
...,...,...
12495,9995,"[[[0.09411764705882353, 0.14509803921568626, 0..."
12496,9996,"[[[0.34802485447304105, 0.6505064721201008, 0...."
12497,9997,"[[[0.4908404181985294, 0.8761029411764709, 0.8..."
12498,9998,"[[[0.3682981004901964, 0.26633731617647094, 0...."


In [65]:
test_images = list(test_data['img'])

In [54]:
test_data.to_csv('test_data.csv')

In [60]:
test_data = pd.read_csv('test_data.csv')

In [66]:
test_data.head()

,id,img
0,1,"[[[0.3803921568627451, 0.2567095588235295, 0.1..."
1,10,"[[[0.0196078431372549, 0.027450980392156862, 0..."
2,100,"[[[0.00784313725490196, 0.00784313725490196, 0..."
3,1000,"[[[0.4701286764705901, 0.6905464920343154, 0.6..."
4,10000,"[[[0.4489123774509831, 0.5104669907513809, 0.5..."


In [67]:
x_test = np.array(np.array(test_images[:]) * 255, dtype = np.uint8)

BATCH_SIZE = 2000
BUFFER_SIZE = BATCH_SIZE * 2
AUTO = tf.data.AUTOTUNE
test_ds = tf.data.Dataset.from_tensor_slices((x_test))
test_ds = test_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTO)

In [68]:
results = model.predict(test_ds)
#test_data.astype(int).sort_values('id', inplace = True)
test_data['label'] = results
#submission['label'] = test_data['label']
#submission['label'] = np.where(submission['label'] <= 0.5,0,1)
test_data[['id','label']].to_csv("submission.csv", index=False)

7/7 [==============================] - 153s 21s/step


In [69]:
test_data.head()

,id,img,label
0,1,"[[[0.3803921568627451, 0.2567095588235295, 0.1...",0.503796
1,10,"[[[0.0196078431372549, 0.027450980392156862, 0...",0.505010
2,100,"[[[0.00784313725490196, 0.00784313725490196, 0...",0.506908
3,1000,"[[[0.4701286764705901, 0.6905464920343154, 0.6...",0.504607
4,10000,"[[[0.4489123774509831, 0.5104669907513809, 0.5...",0.505159
